# Задание 4

In [1]:
import re
import nltk
import json
import math
import string
import pymorphy2
from string import Template
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\пк\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

### Вычисление кол-ва слов на странице

In [2]:
t = Template('{\"page_number\":$page_number,\"words_count\":$words_count}\n')
morph = pymorphy2.MorphAnalyzer(lang='ru')
for i in range(0, 100):
    words = []
    with open('../pages/page№' + str(i + 1) + '.txt', encoding="utf-8") as file:
        page_strip = [row.strip().lower() for row in file]
    words_line = []
    for page_row in page_strip:
        page_row = [token for token in page_row.split() if token not in stopwords.words('russian') and token != " " and token.strip() not in string.punctuation]
        word_symbols = []
        for word in page_row:
            word_symbols.append(word)
        words_line += [word_symbols]   
    for word in words_line:
        words += word
    words = [w for w in filter(re.compile("^[а-яА-Я]+$").match, words)]
    with open('PageWords/pageWords№' + str(i + 1) + '.txt', "w", encoding="utf-8") as tokens_file:
        for token_elem in words:
            tokens_file.write(token_elem + '\n')
    with open('pages_words_count.txt', "a", encoding="utf-8") as counts_file:
        counts_file.write(t.substitute(page_number = i + 1, words_count = len(words)))

### Вычисление кол-ва индексов на страницах

In [3]:
indexes = {}
morph = pymorphy2.MorphAnalyzer(lang='ru')
temp = Template('\'$page_number\': $count')
for i in range(0, 100):
    words = []
    with open('../pages/page№' + str(i + 1) + '.txt', encoding="utf-8") as file:
        page_strip = [row.strip().lower() for row in file]
    words_line = []
    for page_row in page_strip:
        page_row = [token for token in page_row.split() if token not in stopwords.words('russian') and token != " " and token.strip() not in string.punctuation]
        word_symbols = []
        for word in page_row:
            word_symbols.append(word)
        words_line += [word_symbols]   
    for word in words_line:
        words += word
    words = [w for w in filter(re.compile("^[а-яА-Я]+$").match, words)]
    words_indexes = list(dict.fromkeys(words))
    for index in words_indexes:
        if index in indexes.keys():
            indexes[index].append(temp.substitute(page_number = i + 1, count = words.count(index)))
        else:
            indexes[index] = [temp.substitute(page_number = i + 1, count = words.count(index))]

t = Template('{\"count\":$count,\"inverted_array\":{$array},\"word\":\"$word\"}\n')
with open('inverted_index_counts.txt', "w", encoding="utf-8") as inverted_index_counts_file:
    for index_elem in indexes:
        inverted_index_counts_file.write(t.substitute(count = str(len(indexes[index_elem])),
                                          array = ','.join([str(x) for x in indexes[index_elem]]),
                                          word = index_elem))

### Вычисление кол-ва лемм на страницах

In [4]:
with open('inverted_index_counts.txt', encoding="utf-8") as file_with_indexes_counts:
    inverted_index_counts = [row.strip() for row in file_with_indexes_counts]
indexes = {}
for row in inverted_index_counts:
    row = row.replace('\'', '\"')
    jsonIndex = json.loads(row)
    indexes[jsonIndex["word"]] = [jsonIndex["inverted_array"], jsonIndex["count"]]
    
with open('../data/lemms.txt', encoding="utf-8") as file_with_lemms:
    lemms_data = [row.strip() for row in file_with_lemms]
lemms_data_json = {}
for row in lemms_data:
    arr = row.split(':')
    lemms_data_json[arr[0]] = arr[1].strip().split(' ')

t = Template('{\"count\":$count,\"inverted_array\":{$array},\"word\":\"$word\"}\n')
for lemm in lemms_data_json:
    page_counts = {}
    for index in indexes:
        if index in lemms_data_json[lemm]:
            for i in indexes[index][0]:
                if (i in page_counts.keys()):
                    new_count = page_counts.get(i) + indexes[index][0][i]
                    page_counts[i] = new_count
                else:
                    page_counts[i] = indexes[index][0][i]
    with open('lemms_counts.txt', "a", encoding="utf-8") as file_with_lemms_counts:
        file_with_lemms_counts.write(t.substitute(count = str(len(page_counts)),
                                              array = ','.join(["\'" + str(x) + "\'" + ": " + str(page_counts[x]) for x in page_counts]),
                                              word = lemm))

### Вычисления tf idf

In [5]:
termins_tf_idf = []
lemms_tf_idf = []
for i in range(0, 100):
    termins_tf_idf.append({})
    lemms_tf_idf.append({})
    
with open('inverted_index_counts.txt', encoding="utf-8") as file_with_indexes_counts:
    inverted_index_counts = [row.strip() for row in file_with_indexes_counts]
indexes = {}
for row in inverted_index_counts:
    row = row.replace('\'', '\"')
    jsonIndex = json.loads(row)
    indexes[jsonIndex["word"]] = [jsonIndex["count"],jsonIndex["inverted_array"]]
    
with open('lemms_counts.txt', encoding="utf-8") as file_with_lemms_counts:
    lemms_counts = [row.strip() for row in file_with_lemms_counts]
lemms = {}
for row in lemms_counts:
    row = row.replace('\'', '\"')
    jsonCount = json.loads(row)
    lemms[jsonCount["word"]] = [jsonCount["count"], jsonCount["inverted_array"]]
    
with open('pages_words_count.txt', encoding="utf-8") as pages_words_count_file:
    pages_words_counts = [row.strip() for row in pages_words_count_file]
page_words_count_dict = {}
for row in pages_words_counts:
    row = row.replace('\'', '\"')
    jsonCount = json.loads(row)
    page_words_count_dict[jsonCount["page_number"]] = jsonCount["words_count"]

for index in indexes:
    for page_number in indexes[index][1]:
        termins_tf_idf[int(page_number)-1].update({index:[indexes[index][1][page_number]/page_words_count_dict[int(page_number)],
                                  math.log(100/indexes[index][0])]})
for lemm in lemms:
    for page_number in lemms[lemm][1]:
        lemms_tf_idf[int(page_number)-1].update({lemm:[lemms[lemm][1][page_number]/page_words_count_dict[int(page_number)],
                                  math.log(100/lemms[lemm][0])]})

for i in range(0,100):
    with open('Task4data/termins_tf_idf_site№' + str(i + 1) + '.txt', "w", encoding="utf-8") as termins_file:
        for j in termins_tf_idf[i]:
            termins_file.write(j + ' ' + str(termins_tf_idf[i][j][1]) + ' ' + str(termins_tf_idf[i][j][0]) + '\n')
    with open('Task4data/lemms_tf_idf_site№' + str(i + 1) + '.txt', "w", encoding="utf-8") as lemms_file:
        for j in lemms_tf_idf[i]:
            lemms_file.write(j + ' ' + str(lemms_tf_idf[i][j][1]) + ' ' + str(lemms_tf_idf[i][j][0]) + '\n')